In [1]:
import wandb
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

Reusing dataset cnn_dailymail (/home/jvidakovic/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import PegasusTokenizerFast

model_name = "google/pegasus-large"
tokenizer = PegasusTokenizerFast.from_pretrained(model_name)

In [3]:
wandb.init(project="pegasus-cnn-fine-tuning", entity="jankovidakovic")


wandb: Currently logged in as: jankovidakovic (use `wandb login --relogin` to force relogin)


In [4]:
from utils import prepare_dataset

eval_dataset = prepare_dataset(
    dataset["validation"]["article"],
    dataset["validation"]["highlights"],
    tokenizer=tokenizer
)


In [5]:
train_dataset = prepare_dataset(
    dataset["train"]["article"],
    dataset["train"]["highlights"],
    tokenizer=tokenizer
)

In [13]:
item = train_dataset[0]
type(item)

dict

In [14]:
item.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [15]:
item["labels"]

tensor([10298,  1571,   151,  ...,     0,     0,     0])

In [7]:
from utils import prepare_model

model, optimizer, scheduler = prepare_model(device="cuda")

In [8]:
from transformers import TrainingArguments, IntervalStrategy

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,  # output directory
    per_device_train_batch_size=1,  # batch size per device during training, can increase if memory allows
    per_device_eval_batch_size=1,  # batch size for evaluation, can increase if memory allows
    save_steps=500,  # number of updates steps before checkpoint saves
    save_total_limit=3,  # limit the total amount of checkpoints and deletes the older checkpoints
    evaluation_strategy=IntervalStrategy.STEPS,  # evaluation strategy to adopt during training
    eval_steps=100,  # number of update steps before evaluation
    # warmup_steps=500,  # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=10,
    #####
    label_smoothing_factor=0.1,
    max_steps=170000,
    # per_device_train_batch_size=1,  # total batch size = 2
    gradient_accumulation_steps=128,   # batch_size_orig / batch_size_possible
    report_to=["wandb"],
)

In [16]:
from transformers import Trainer, PegasusTokenizer

trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # tokenizer=tokenizer,
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large"),
    model=model,
    # compute_metrics="hehe"  # i suppose computing rouge on every step is overkill
    optimizers=(optimizer, scheduler),
    args=training_args
)

loading file https://huggingface.co/google/pegasus-large/resolve/main/spiece.model from cache at /home/jvidakovic/.cache/huggingface/transformers/66f187d645734a6204f3fd24593fbf0d9e36b528dd85b3adae9a566b17b4768f.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8
loading file https://huggingface.co/google/pegasus-large/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/pegasus-large/resolve/main/special_tokens_map.json from cache at /home/jvidakovic/.cache/huggingface/transformers/fbf9c7cf2d49b24712b53a2760e7c62a2acecd1496908822df00b8ec2683ca6d.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
loading file https://huggingface.co/google/pegasus-large/resolve/main/tokenizer_config.json from cache at /home/jvidakovic/.cache/huggingface/transformers/74256fafbb3cb536e351e6731914d42f732e77d33e537b6c19fb72f4b74f50ea.43f396f0ee3b974f9128267d49f69a26b11f3ed290851ac5788a549cc2979671
loading configuration file https://huggingfac

In [17]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 287113
  Num Epochs = 152
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 128
  Total optimization steps = 170000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/jvidakovic/.conda/envs/seminar-2/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/jvidakovic/.conda/envs/seminar-2/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jvidakovic/.conda/envs/seminar-2/lib/python3.9/site-packages/transformers/models/pegasus/modeling_pegasus.py", line 1392, in forward
    outputs = self.model(
  File "/home/jvidakovic/.conda/envs/seminar-2/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jvidakovic/.conda/envs/seminar-2/lib/python3.9/site-packages/transformers/models/pegasus/modeling_pegasus.py", line 1249, in forward
    decoder_outputs = self.decoder(
  File "/home/jvidakovic/.conda